In [ ]:
# PART 1: GENERATE DATA
from stockfish import Stockfish
import chess
import random

"""def generate_random_board():
    board = chess.Board()
    for _ in range(random.randint(10, 40)):  # Adjust range to vary complexity
        legal_moves = list(board.legal_moves)
        if not legal_moves:
            break
        move = random.choice(legal_moves)
        board.push(move)
    return board
"""
# TRYING TO BALANCE THE TRAINING DATA QUALITY
def is_balanced(board):
    piece_counts = {chess.WHITE: 0, chess.BLACK: 0}
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            piece_counts[piece.color] += 1
    white_pieces = piece_counts[chess.WHITE]
    black_pieces = piece_counts[chess.BLACK]
    return abs(white_pieces - black_pieces) <= 3
def generate_random_board():
    board = chess.Board()
    for _ in range(random.randint(10, 40)):
        legal_moves = list(board.legal_moves)
        if not legal_moves:
            break
        move = random.choice(legal_moves)
        board.push(move)
        if not is_balanced(board):
            break
    return board

engine = Stockfish("C:\\Users\\16812\\OneDrive\\Desktop\\stockfish\\stockfish-windows-x86-64-avx2.exe")
def generate_dataset(num_positions=10000):
    dataset = []
    for i in range(num_positions):
        board = generate_random_board()
        best_move = engine.get_best_move()
        evaluation = engine.get_evaluation()
        if best_move and evaluation:
            dataset.append({
                "FEN": board.fen(),
                "Best Move": best_move,
                "Evaluation": evaluation["value"]
            })
    return dataset
print("Generating Data...")
chess_data = generate_dataset()
print("Finished generating")

Generating Data...
Finished generating


In [75]:
# PART 2: MAKE THE MODEL

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

def fen_to_vector(fen):
    board = chess.Board(fen)
    vector = np.zeros(64, dtype=np.float32)
    for square, piece in board.piece_map().items():
        vector[square] = piece.piece_type
    return vector

dataset = chess_data
X = np.array([fen_to_vector(entry["FEN"]) for entry in dataset])
y = np.array([entry["Evaluation"] for entry in dataset])
model = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(len(X[0]),)),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="linear")
])
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
print("Training model...")
model.fit(X, y, epochs=100, batch_size=16)

Training model...
Epoch 1/100
625/625 [==============================] - 1s 1ms/step - loss: 44.7481 - mae: 5.0998
Epoch 2/100
625/625 [==============================] - 1s 1ms/step - loss: 30.8120 - mae: 4.4967
Epoch 3/100
625/625 [==============================] - 1s 1ms/step - loss: 30.6186 - mae: 4.4857
Epoch 4/100
625/625 [==============================] - 1s 1ms/step - loss: 30.3802 - mae: 4.4579
Epoch 5/100
625/625 [==============================] - 1s 1ms/step - loss: 30.0791 - mae: 4.4426
Epoch 6/100
625/625 [==============================] - 1s 1ms/step - loss: 29.3685 - mae: 4.3877
Epoch 7/100
625/625 [==============================] - 1s 1ms/step - loss: 28.8865 - mae: 4.3499
Epoch 8/100
625/625 [==============================] - 1s 1ms/step - loss: 28.4161 - mae: 4.3003
Epoch 9/100
625/625 [==============================] - 1s 1ms/step - loss: 27.9048 - mae: 4.2642
Epoch 10/100
625/625 [==============================] - 1s 1ms/step - loss: 27.4096 - mae: 4.2270
Epoch 11/10

In [78]:
# PART 3: MAKE A POSITION TO EVAULATE
def is_balanced(board):
    piece_counts = {chess.WHITE: 0, chess.BLACK: 0}
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            piece_counts[piece.color] += 1
    white_pieces = piece_counts[chess.WHITE]
    black_pieces = piece_counts[chess.BLACK]
    return abs(white_pieces - black_pieces) <= 3

def generate_random_board():
    board = chess.Board()
    for _ in range(random.randint(10, 40)):
        legal_moves = list(board.legal_moves)
        if not legal_moves:
            break
        move = random.choice(legal_moves)
        board.push(move)
        if not is_balanced(board):
            break
    return board.fen()

# Example usage
random_fen = generate_random_board()
print("Random FEN:", random_fen)

Random FEN: rn1qk1nr/p2p1p2/b1p5/1pb1p1Pp/3PB2R/5NP1/PPP1PP2/RNBQ1K2 w kq - 4 10


In [ ]:
# PART 4: TEST THE MODEL
test_vector = fen_to_vector(random_fen).reshape(1, -1)
predicted_eval = model.predict(test_vector)
print(f"My AI Evaluation: {predicted_eval[0][0]}")
engine.set_fen_position(random_fen)
stockfish_eval = engine.get_evaluation()
eval_value = stockfish_eval.get("value", "N/A")
print(f"Stockfish Evaluation: {eval_value}")

1/1 [==============================] - 0s 20ms/step
My AI Evaluation: 29.566890716552734
Stockfish Evaluation: 629
